# **1-Library Installation and Initial Setup**

In [ ]:
%%capture
!pip install diffusers transformers accelerate
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121
!pip install pillow matplotlib ipywidgets

# **2-Imports and Setup**

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
from io import BytesIO
from google.colab import files

# **3-GPU Configuration**

In [ ]:
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.benchmark = True

# **4-Model Loading**

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False
).to("cuda")

pipe.enable_attention_slicing()

# **5-Steganography Class**

In [ ]:
class AdvancedSteganography:
    def __init__(self):
        self.max_data_size = 1024  # Increase storage capacity

    def _complex_encode(self, img_array, data):
        binary_data = ''.join(format(ord(c), '08b') for c in data)
        length_info = format(len(binary_data), '016b')
        binary_data = length_info + binary_data

        if len(binary_data) > self.max_data_size:
            raise ValueError(f"Data is too large (max {self.max_data_size//8} characters)")

        for i, bit in enumerate(binary_data):
            x = (i * 13) % img_array.shape[0]
            y = (i * 19) % img_array.shape[1]
            if i < 16:
                channel = i % 3
            else:
                channel = (i - 16) % 3
            img_array[x, y, channel] = (img_array[x, y, channel] & 0xFE) | int(bit)

        return img_array

    def encode(self, image, secret_data):
        img_array = np.array(image)
        if img_array.shape[2] > 3:
            img_array = img_array[..., :3]
        if len(secret_data) == 0:
            raise ValueError("Secret data cannot be empty")
        encoded_array = self._complex_encode(img_array.copy(), secret_data)
        return Image.fromarray(encoded_array)

    def decode(self, image):
        img_array = np.array(image)
        if img_array.shape[2] > 3:
            img_array = img_array[..., :3]
        binary_data = []
        length_bits = []
        for i in range(16):
            x = (i * 13) % img_array.shape[0]
            y = (i * 19) % img_array.shape[1]
            channel = i % 3
            bit = img_array[x, y, channel] & 1
            length_bits.append(str(bit))
        data_length = int(''.join(length_bits), 2)
        for i in range(16, 16 + data_length):
            x = (i * 13) % img_array.shape[0]
            y = (i * 19) % img_array.shape[1]
            channel = (i - 16) % 3
            bit = img_array[x, y, channel] & 1
            binary_data.append(str(bit))
        text = ''.join(chr(int(''.join(binary_data[i:i+8]), 2)) for i in range(0, len(binary_data), 8))
        return text

# **6-UI Components**

In [ ]:
style = {'description_width': '150px'}
layout = widgets.Layout(width='auto')

secret_input = widgets.Textarea(
    value='Your secret message here...',
    description='Secret message:',
    style=style,
    layout=layout
)

prompt_input = widgets.Text(
    value='A cyberpunk city at night, 4k detailed',
    description='Image prompt:',
    style=style,
    layout=layout
)

encode_btn = widgets.Button(description="Encode Image", button_style='success')
decode_btn = widgets.Button(description="Reveal Secret from File", button_style='info')
decode_from_memory_btn = widgets.Button(description="Reveal Secret from Generated Image", button_style='warning')
output = widgets.Output()

# **7-Event Handlers**

In [ ]:
encoded_img_memory = {'img': None}

def on_encode(b):
    with output:
        output.clear_output()
        print("🛠️ Generating the magic image...")
        try:
            image = pipe(prompt_input.value, num_inference_steps=30).images[0]
            steg = AdvancedSteganography()
            encoded_img = steg.encode(image, secret_input.value)
            encoded_img_memory['img'] = encoded_img  # Save image in memory

            plt.figure(figsize=(15, 7))
            plt.subplot(1, 2, 1)
            plt.imshow(image)
            plt.title("Original Image", fontsize=12, pad=10)
            plt.axis('off')

            plt.subplot(1, 2, 2)
            plt.imshow(encoded_img)
            plt.title("Image with Secret", fontsize=12, pad=10)
            plt.axis('off')

            plt.tight_layout()
            plt.show()

            encoded_img.save("secret_image.png")
            print("✅ Encoded image saved! (secret_image.png)")
            print(f"📏 Secret data size: {len(secret_input.value)} characters")
            display(decode_from_memory_btn)  # Show the button to reveal secret from generated image
        except Exception as e:
            print(f"❌ Encoding error: {str(e)}")

def on_decode(b):
    with output:
        output.clear_output()
        print("🔍 Revealing the secret from file...")
        try:
            uploaded = files.upload()
            if not uploaded:
                print("❌ No image uploaded")
                return
            img = Image.open(BytesIO(uploaded[next(iter(uploaded))]))
            steg = AdvancedSteganography()
            secret = steg.decode(img)
            print("\n🔥 Secret revealed!")
            display(widgets.HTML(f'<div style="font-size:20px; color:#ff00ff; padding:10px; border:2px dashed #00ff00">{secret}</div>'))
        except Exception as e:
            print(f"❌ Decoding error: {str(e)}")

def on_decode_from_memory(b):
    with output:
        output.clear_output()
        print("🔍 Revealing the secret from the generated image...")
        if encoded_img_memory['img'] is None:
            print("❌ Please generate an encoded image first.")
            return
        try:
            steg = AdvancedSteganography()
            secret = steg.decode(encoded_img_memory['img'])
            print("\n🔥 Secret revealed!")
            display(widgets.HTML(f'<div style="font-size:20px; color:#ff00ff; padding:10px; border:2px dashed #00ff00">{secret}</div>'))
        except Exception as e:
            print(f"❌ Decoding error: {str(e)}")

# **8-Main UI Assembly**

In [ ]:
encode_btn.on_click(on_encode)
decode_btn.on_click(on_decode)
decode_from_memory_btn.on_click(on_decode_from_memory)

display(widgets.VBox([
    widgets.HTML("<h1 style='color:#00ffff;'>🕵️‍♂️ Advanced Steganography System</h1>"),
    widgets.HTML("<p style='color:#aaaaaa;'>Generate an image and hide your secret in it!</p>"),
    prompt_input,
    secret_input,
    widgets.HBox([encode_btn, decode_btn]),
    output
]))